# 3.0.0 Export

- En este archivo se exportara la informacion necesaria para el dash

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.RFM import rfm_v1, rfm_v2
from utils.paths2 import direcciones_cursos

G_raw, G_processed, G_interim, G_external, G_models, G_reports, G_reports_figures = direcciones_cursos('australian')

In [4]:
df = pd.read_csv(G_raw/'sample.csv')
df['date'] =  pd.to_datetime(df['date'])
df.head()

,invoice,date,customer_id,stockcode,price
0,1,2020-07-25,d18734,y7,50.45
1,2,2020-01-17,c21086,x7,25.30
2,3,2019-07-05,d18185,z5,18.40
3,4,2019-02-26,c18331,z2,5.50
4,5,2019-02-10,b16309,y7,18.40


In [5]:
df_rfm = pd.read_parquet(G_models/'df_rfm.parquet.gzip')
df_ks = pd.read_parquet(G_models/'df_ks.parquet.gzip')

In [6]:
df.shape, df_rfm.shape, df_ks.shape

((100000, 5), (33004, 9), (33004, 3))

In [7]:
# joins
df = pd.merge(df, df_rfm, how='left', on=['customer_id'])  
df = pd.merge(df, df_ks, how='left', on=['customer_id'])
df.shape

(100000, 15)

In [8]:
df.head()

,invoice,date,customer_id,stockcode,price,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,rfm_score,group,Cluster,Cluster_medoids
0,1,2020-07-25,d18734,y7,50.45,153,5,169.84,3,4,5,4.000000,7,0,0
1,2,2020-01-17,c21086,x7,25.30,10,3,81.25,5,3,4,4.000000,7,2,3
2,3,2019-07-05,d18185,z5,18.40,141,7,94.26,3,4,4,3.666667,6,0,0
3,4,2019-02-26,c18331,z2,5.50,40,3,56.10,5,3,3,3.666667,6,2,3
4,5,2019-02-10,b16309,y7,18.40,455,2,23.90,1,2,1,1.333333,1,1,2


In [9]:
# save
df.to_parquet(G_processed/'data.parquet.gzip', compression='gzip')